# Clean Building Inventory
Step required to make the Probabilistic Housing Unit Allocation work.

The Building Foot Print Data for Joplin Needs to be Merged with the Following:
    
    1. Read in Building Inventory
    2. Check to make sure building inventory has a unique id
    3. Add representative point and polygon WKTs
    4. Add Census Block ID
    5. Add Parcel ID
    

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import pysal as ps
import math as math
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely
import descartes

import folium as fm # folium has more dynamic maps - but requires internet connection

import os # For saving output to path

C:\Users\nathanael99\AppData\Local\Continuum\anaconda3\envs\pyincore20190618\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
# Display versions being used - important information for replication
import sys
print("Python Version     ", sys.version)
print("pysal version:     ", ps.__version__)
print("numpy version:     ", np.__version__)
print("geopandas version: ", gpd.__version__)
print("pandas version:    ", pd.__version__)
print("shapely version:   ", shapely.__version__)
# print("descartes version:   ", descartes.__version__)  1.1.0
print("folium version:    ", fm.__version__)

Python Version      3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
pysal version:      2.0.0
numpy version:      1.16.4
geopandas version:  0.5.0
pandas version:     0.24.2
shapely version:    1.6.4.post1
folium version:     0.9.1


In [3]:
# Store Program Name for output files to have the same name
programname = "IN-CORE_1cv2_Joplin_CleanBuildingInventory_2019-07-13"
# Make directory to save output
if not os.path.exists(programname):
    os.mkdir(programname)

## Read in Building Footprints
The building footprints provide basic understanding of where address points can be located.

In [4]:
# building_footprints_shp = '../../SourceData/joplin_footprints/Attary_Joplin_Building_Footprint_2016-08-30.shp'
building_footprints_shp = '../../SourceData/joplin_footprints/confuence_joplin_datasets/joplin-bldg-footprint/joplin_bldg.shp'
building_gdf = gpd.read_file(building_footprints_shp)
building_gdf.crs

{'init': 'epsg:4326'}

In [5]:
building_gdf.head()

,ADDRESS1,HOUSE_NO,archtype,objectid,parid,parid_card,bldg_id,struct_typ,str_prob,year_built,...,cont_val,efacility,dwell_unit,str_typ2,occ_typ2,tract_id,guid,x,y,geometry
0,2519 N HIGHVIEW AVE,2519,1,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477334,37.111732,POLYGON ((-94.47739972933748 37.11177310447127...
1,2515 N HIGHVIEW AVE,2515,1,2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477190,37.111399,"POLYGON ((-94.4771326960593 37.11131771353134,..."
2,2511 N HIGHVIEW AVE,2511,1,3,None,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.476820,37.111433,POLYGON ((-94.47689239616878 37.11139956669546...
3,2511 N HIGHVIEW AVE,2511,1,4,None,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477223,37.111174,POLYGON ((-94.47727628758035 37.11108068083621...
4,2501 N HIGHVIEW AVE,2501,1,5,None,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477309,37.110922,"POLYGON ((-94.47736583467571 37.1108311087985,..."


In [6]:
# lok at Archtypes
pd.crosstab(index=building_gdf['archtype'], columns="count")

col_0,count
archtype,
1,24762
5,147
6,736
7,964
8,155
9,39
10,50
11,8
12,41


## Create Unique Building ID
Building ID will be important for linking Address Point Inventory to Buildings and Critical Infrastructure Inventories to Buildings.

ID must be unique and non-missing.

In [7]:
# Count the number of Unique Values
building_gdf[['objectid']].describe()

,objectid
count,28160.000000
mean,16004.287393
std,9186.240385
min,1.000000
25%,7927.750000
50%,16581.500000
75%,23876.250000
max,31544.000000


In [8]:
# Count the number of Unique Values
building_gdf[['objectid']].nunique()

objectid    28152
dtype: int64

In [9]:
# Are there any missing values for the unique id?
building_gdf.loc[building_gdf['objectid'].isnull()]

,ADDRESS1,HOUSE_NO,archtype,objectid,parid,parid_card,bldg_id,struct_typ,str_prob,year_built,...,cont_val,efacility,dwell_unit,str_typ2,occ_typ2,tract_id,guid,x,y,geometry


In [10]:
# List duplicates for the Unique ID
pd.crosstab(index=building_gdf.duplicated(subset=['objectid']), columns="count", margins=True, margins_name="Total")

col_0,count,Total
row_0,,
False,28152,28152
True,8,8
Total,28160,28160


In [11]:
# Create Building ID String based on ObjectID
building_gdf['bldg_id'] = building_gdf['objectid'].apply(lambda x : "B"+str(int(x)).zfill(7))
building_gdf[['objectid', 'bldg_id']].head().append(building_gdf[['objectid', 'bldg_id']].tail())

,objectid,bldg_id
0,1,B0000001
1,2,B0000002
2,3,B0000003
3,4,B0000004
4,5,B0000005
28155,31537,B0031537
28156,12792,B0012792
28157,12780,B0012780
28158,12789,B0012789
28159,31543,B0031543


In [12]:
# Move Primary Key Column Building ID to first Column
cols = ['bldg_id']  + [col for col in building_gdf if col != 'bldg_id']
cols
building_gdf = building_gdf[cols]
building_gdf.head()

,bldg_id,ADDRESS1,HOUSE_NO,archtype,objectid,parid,parid_card,struct_typ,str_prob,year_built,...,cont_val,efacility,dwell_unit,str_typ2,occ_typ2,tract_id,guid,x,y,geometry
0,B0000001,2519 N HIGHVIEW AVE,2519,1,1,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477334,37.111732,POLYGON ((-94.47739972933748 37.11177310447127...
1,B0000002,2515 N HIGHVIEW AVE,2515,1,2,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477190,37.111399,"POLYGON ((-94.4771326960593 37.11131771353134,..."
2,B0000003,2511 N HIGHVIEW AVE,2511,1,3,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.476820,37.111433,POLYGON ((-94.47689239616878 37.11139956669546...
3,B0000004,2511 N HIGHVIEW AVE,2511,1,4,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477223,37.111174,POLYGON ((-94.47727628758035 37.11108068083621...
4,B0000005,2501 N HIGHVIEW AVE,2501,1,5,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.477309,37.110922,"POLYGON ((-94.47736583467571 37.1108311087985,..."


In [13]:
# Confirm Building ID is Unique and Non-Missing
building_gdf[['bldg_id']].describe()

,bldg_id
count,28160
unique,28152
top,B0027527
freq,2


In [14]:
# List duplicates for the Unique ID
building_gdf.loc[building_gdf.duplicated(subset=['bldg_id'])==True]

,bldg_id,ADDRESS1,HOUSE_NO,archtype,objectid,parid,parid_card,struct_typ,str_prob,year_built,...,cont_val,efacility,dwell_unit,str_typ2,occ_typ2,tract_id,guid,x,y,geometry
10513,B0000003,2511 N HIGHVIEW AVE,2511,5,3,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.476820,37.111433,POLYGON ((-94.47689239616878 37.11139956669546...
12261,B0021351,1008 E HILL ST,1008,7,21351,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.501833,37.092350,POLYGON ((-94.50192031710587 37.09230885292315...
12340,B0023016,6032 W HIGHLAND DR,6032,1,23016,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.492760,37.021306,POLYGON ((-94.49263782542808 37.02129054064547...
13415,B0030178,3125 E 6TH ST,3125,15,30178,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.475855,37.085231,POLYGON ((-94.47600390305914 37.08502900032955...
13552,B0001541,402 E 44TH ST,402,1,1541,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.510159,37.040360,POLYGON ((-94.51028541806247 37.04028557397738...
13825,B0001995,2427 S CLEVELAND AVE,2427,1,1995,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.554072,37.064733,"POLYGON ((-94.55413407318056 37.0646510008912,..."
25994,B0027527,None,None,1,27527,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.540667,37.067841,POLYGON ((-94.54062833895729 37.06787714113283...
26447,B0028029,2715 OLIVER AVE,2715,1,28029,None,None,None,None,None,...,None,None,None,None,None,None,None,-94.540698,37.060290,POLYGON ((-94.54064264914901 37.06025062848077...


In [15]:
building_gdf = building_gdf.drop_duplicates(subset=['bldg_id'])

In [16]:
# Confirm Building ID is Unique and Non-Missing
building_gdf[['bldg_id']].describe()

,bldg_id
count,28152
unique,28152
top,B0023070
freq,1


In [17]:
# Are there any missing values for the unique id?
building_gdf.loc[building_gdf['bldg_id'].isnull()]

,bldg_id,ADDRESS1,HOUSE_NO,archtype,objectid,parid,parid_card,struct_typ,str_prob,year_built,...,cont_val,efacility,dwell_unit,str_typ2,occ_typ2,tract_id,guid,x,y,geometry


### Identify Representative Point for Each Building Foot Print 
Will need to find the internal or representive point inside the polygon.
Then identify the Census Block for each building based on the internal point.

The command

ref_polygon.representative_point().wkt 

https://gis.stackexchange.com/questions/43384/python-find-a-method-to-calculate-the-inner-centroid-also-known-as-labelpoin



In [18]:
# Building Data projected to have lat Lon for building rep point
latlong_crs = {'init':'epsg:4326'}
building_gdf = building_gdf.to_crs(latlong_crs)

In [19]:
# Add Representative Point
building_gdf.loc[building_gdf.index,'rppnt4326'] = building_gdf['geometry'].representative_point()
building_gdf['rppnt4326'].label = "Representative Point EPSG 4326 (WKT)"
building_gdf['rppnt4326'].notes = "Internal Point within building footprint"

# Add Column that Duplicates Polygon Geometry
building_gdf.loc[building_gdf.index,'bply4326'] = building_gdf['geometry']
building_gdf['bply4326'].label = "Footprint Polygon EPSG 4326 (WKT)"
building_gdf['bply4326'].notes = "Polygon Shape Points for Foot Print - drawn by Navid Attary CSU 2016"

In [20]:
building_gdf.head()

,bldg_id,ADDRESS1,HOUSE_NO,archtype,objectid,parid,parid_card,struct_typ,str_prob,year_built,...,dwell_unit,str_typ2,occ_typ2,tract_id,guid,x,y,geometry,rppnt4326,bply4326
0,B0000001,2519 N HIGHVIEW AVE,2519,1,1,None,None,None,None,None,...,None,None,None,None,None,-94.477334,37.111732,POLYGON ((-94.47739972933748 37.11177310447127...,POINT (-94.47732618279426 37.1117474687169),POLYGON ((-94.47739972933748 37.11177310447127...
1,B0000002,2515 N HIGHVIEW AVE,2515,1,2,None,None,None,None,None,...,None,None,None,None,None,-94.477190,37.111399,"POLYGON ((-94.4771326960593 37.11131771353134,...",POINT (-94.47719019368949 37.1114010706726),"POLYGON ((-94.4771326960593 37.11131771353134,..."
2,B0000003,2511 N HIGHVIEW AVE,2511,1,3,None,None,None,None,None,...,None,None,None,None,None,-94.476820,37.111433,POLYGON ((-94.47689239616878 37.11139956669546...,POINT (-94.47682036688217 37.11143251198735),POLYGON ((-94.47689239616878 37.11139956669546...
3,B0000004,2511 N HIGHVIEW AVE,2511,1,4,None,None,None,None,None,...,None,None,None,None,None,-94.477223,37.111174,POLYGON ((-94.47727628758035 37.11108068083621...,POINT (-94.47722272093182 37.11117421422948),POLYGON ((-94.47727628758035 37.11108068083621...
4,B0000005,2501 N HIGHVIEW AVE,2501,1,5,None,None,None,None,None,...,None,None,None,None,None,-94.477309,37.110922,"POLYGON ((-94.47736583467571 37.1108311087985,...",POINT (-94.47730922150966 37.11092173624859),"POLYGON ((-94.47736583467571 37.1108311087985,..."


### Calculate Ground Square Feet and Square Meters for each footprint

In [21]:
# Building Data projected to UTM 
# calculate the UTM zone from this avg longitude and define the UTM
# CRS to project
avg_longitude = building_gdf['geometry'].unary_union.centroid.x

utm_zone = int(math.floor((avg_longitude + 180) / 6.) + 1)
utm_crs = {'datum': 'NAD83',
           'ellps': 'GRS80',
           'proj' : 'utm',
           'zone' : utm_zone,
           'units': 'm'}
building_gdf_utm = building_gdf.to_crs(utm_crs)
building_gdf_utm[['bldg_id','geometry','bply4326']].head()

,bldg_id,geometry,bply4326
0,B0000001,"POLYGON ((368735.4543178821 4108293.09350737, ...",POLYGON ((-94.47739972933748 37.11177310447127...
1,B0000002,"POLYGON ((368758.3951430458 4108242.200750984,...","POLYGON ((-94.4771326960593 37.11131771353134,..."
2,B0000003,"POLYGON ((368779.8880900628 4108250.949809433,...",POLYGON ((-94.47689239616878 37.11139956669546...
3,B0000004,"POLYGON ((368745.2272216786 4108216.101569349,...",POLYGON ((-94.47727628758035 37.11108068083621...
4,B0000005,"POLYGON ((368736.8396840204 4108188.536496918,...","POLYGON ((-94.47736583467571 37.1108311087985,..."


In [22]:
building_gdf_utm['gsq_meter'] = building_gdf_utm['geometry'].area
building_gdf_utm[['bldg_id','geometry','bply4326','gsq_meter']].head()

,bldg_id,geometry,bply4326,gsq_meter
0,B0000001,"POLYGON ((368735.4543178821 4108293.09350737, ...",POLYGON ((-94.47739972933748 37.11177310447127...,107.940427
1,B0000002,"POLYGON ((368758.3951430458 4108242.200750984,...","POLYGON ((-94.4771326960593 37.11131771353134,...",169.494274
2,B0000003,"POLYGON ((368779.8880900628 4108250.949809433,...",POLYGON ((-94.47689239616878 37.11139956669546...,93.622453
3,B0000004,"POLYGON ((368745.2272216786 4108216.101569349,...",POLYGON ((-94.47727628758035 37.11108068083621...,185.178358
4,B0000005,"POLYGON ((368736.8396840204 4108188.536496918,...","POLYGON ((-94.47736583467571 37.1108311087985,...",202.291430


In [23]:
building_gdf_utm[['gsq_meter']].describe()

,gsq_meter
count,28152.000000
mean,257.480731
std,910.152692
min,0.000336
25%,104.809298
50%,162.019197
75%,240.030400
max,90132.331610


In [24]:
building_gdf_utm.crs

{'datum': 'NAD83', 'ellps': 'GRS80', 'proj': 'utm', 'zone': 15, 'units': 'm'}

In [25]:
# Add ground square meters to building goedataframe
building_gdf = pd.merge(building_gdf, building_gdf_utm[['bldg_id','gsq_meter']], 
                        left_on='bldg_id', right_on='bldg_id', how='left')
building_gdf[['bldg_id','geometry','bply4326','gsq_meter']].head()

,bldg_id,geometry,bply4326,gsq_meter
0,B0000001,POLYGON ((-94.47739972933748 37.11177310447127...,POLYGON ((-94.47739972933748 37.11177310447127...,107.940427
1,B0000002,"POLYGON ((-94.4771326960593 37.11131771353134,...","POLYGON ((-94.4771326960593 37.11131771353134,...",169.494274
2,B0000003,POLYGON ((-94.47689239616878 37.11139956669546...,POLYGON ((-94.47689239616878 37.11139956669546...,93.622453
3,B0000004,POLYGON ((-94.47727628758035 37.11108068083621...,POLYGON ((-94.47727628758035 37.11108068083621...,185.178358
4,B0000005,"POLYGON ((-94.47736583467571 37.1108311087985,...","POLYGON ((-94.47736583467571 37.1108311087985,...",202.291430


## Read in Census Block Data
Census Blocks provide an estimate of how many residiential address points (housing units) should be located in each block.

In [26]:
source_program = 'IN-CORE_1av2_Joplin_CleanBlockData_2019-07-10'
census_blocks_csv = source_program+"/"+source_program+"EPSG4269.csv"
census_blocks_df = pd.read_csv(census_blocks_csv)
census_blocks_gdf = gpd.GeoDataFrame(census_blocks_df)
census_blocks_gdf.head()

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,...,blk104269,blockid,apcount,pop10,gqpop10,popdiff,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10
0,0,29,97,12100,1047,290970121001047,N,2,4,"POLYGON ((-94.13775 37.32550000000001, -94.138...",...,"POLYGON ((-94.13775 37.32550000000001, -94.138...",2.909701e+14,2.0,4.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
1,1,29,97,12100,1050,290970121001050,N,3,7,"POLYGON ((-94.137637 37.328675, -94.119315 37....",...,"POLYGON ((-94.137637 37.328675, -94.119315 37....",2.909701e+14,3.0,7.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
2,2,29,97,12100,1094,290970121001094,N,4,13,"POLYGON ((-94.214761 37.293836, -94.230751 37....",...,"POLYGON ((-94.214761 37.293836, -94.230751 37....",2.909701e+14,4.0,13.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
3,3,29,97,12100,1093,290970121001093,N,5,5,"POLYGON ((-94.197294 37.27723599999999, -94.19...",...,"POLYGON ((-94.197294 37.27723599999999, -94.19...",2.909701e+14,5.0,5.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
4,4,29,97,12100,1130,290970121001130,N,4,9,"POLYGON ((-94.151792 37.276275, -94.1519139999...",...,"POLYGON ((-94.151792 37.276275, -94.1519139999...",2.909701e+14,4.0,9.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA


In [27]:
census_blocks_gdf.columns

Index(['Unnamed: 0', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE',
       'BLOCKID10', 'PARTFLG', 'HOUSING10', 'POP10', 'geometry',
       'CountySelect', 'rppnt4269', 'blk104269', 'blockid', 'apcount', 'pop10',
       'gqpop10', 'popdiff', 'PLCGEOID10', 'PLCNAME10', 'PUMGEOID10',
       'PUMNAME10'],
      dtype='object')

In [28]:
census_blocks_gdf['geometry'].geom_type.describe()

count     0
unique    0
dtype: int64

In [29]:
# Use shapely.wkt loads to convert WKT to GeoSeries
from shapely.wkt import loads

census_blocks_gdf['geometry'] = census_blocks_gdf['geometry'].apply(lambda x: loads(x))
census_blocks_gdf['geometry'].geom_type.describe()

count        9621
unique          2
top       Polygon
freq         9615
dtype: object

In [30]:
census_blocks_gdf = census_blocks_gdf.set_geometry(census_blocks_gdf['geometry'])
census_blocks_gdf.crs = {'init':'epsg:4269'}
census_blocks_gdf.head()

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,...,blk104269,blockid,apcount,pop10,gqpop10,popdiff,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10
0,0,29,97,12100,1047,290970121001047,N,2,4,"POLYGON ((-94.13775 37.32550000000001, -94.138...",...,"POLYGON ((-94.13775 37.32550000000001, -94.138...",2.909701e+14,2.0,4.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
1,1,29,97,12100,1050,290970121001050,N,3,7,"POLYGON ((-94.137637 37.328675, -94.119315 37....",...,"POLYGON ((-94.137637 37.328675, -94.119315 37....",2.909701e+14,3.0,7.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
2,2,29,97,12100,1094,290970121001094,N,4,13,"POLYGON ((-94.214761 37.293836, -94.230751 37....",...,"POLYGON ((-94.214761 37.293836, -94.230751 37....",2.909701e+14,4.0,13.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
3,3,29,97,12100,1093,290970121001093,N,5,5,"POLYGON ((-94.197294 37.27723599999999, -94.19...",...,"POLYGON ((-94.197294 37.27723599999999, -94.19...",2.909701e+14,5.0,5.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA
4,4,29,97,12100,1130,290970121001130,N,4,9,"POLYGON ((-94.151792 37.276275, -94.1519139999...",...,"POLYGON ((-94.151792 37.276275, -94.1519139999...",2.909701e+14,4.0,9.0,0.0,0.0,NaN,NaN,2902800,Jasper & Newton Counties PUMA


In [31]:
# Check CRS for Building Centroid and Block
census_blocks_gdf.crs

{'init': 'epsg:4269'}

In [32]:
building_gdf.crs

{'init': 'epsg:4326'}

In [33]:
# Convert Census Block CRS to Buildings CRS
census_blocks_gdf = census_blocks_gdf.to_crs(building_gdf.crs)
census_blocks_gdf.crs

{'init': 'epsg:4326'}

In [34]:
# Check change in Geometry
census_blocks_gdf['blk104269'] = census_blocks_gdf['blk104269'].apply(lambda x: loads(x))
census_blocks_gdf[['geometry','blk104269']].loc[census_blocks_gdf['geometry'] != census_blocks_gdf['blk104269']]

,geometry,blk104269


In [35]:
census_blocks_gdf[['geometry','blk104269']].head()

,geometry,blk104269
0,"POLYGON ((-94.13775 37.32550000000001, -94.138...","POLYGON ((-94.13775 37.32550000000001, -94.138..."
1,"POLYGON ((-94.137637 37.328675, -94.119315 37....","POLYGON ((-94.137637 37.328675, -94.119315 37...."
2,"POLYGON ((-94.214761 37.293836, -94.230751 37....","POLYGON ((-94.214761 37.293836, -94.230751 37...."
3,"POLYGON ((-94.197294 37.27723599999999, -94.19...","POLYGON ((-94.197294 37.27723599999999, -94.19..."
4,"POLYGON ((-94.151792 37.276275, -94.1519139999...","POLYGON ((-94.151792 37.276275, -94.1519139999..."


In [36]:
census_blocks_gdf.geometry.name

'geometry'

### Need to explore projection issues
It looks like NAD 83 (EPSG 4326) and WGS 84 (EPSG 4269) Produce the same lat lan coordinates. I was expecting there to be slight differences.

In [37]:
# Convert BLOCKID10 to a string
census_blocks_gdf['BLOCKID10'] = census_blocks_gdf['BLOCKID10'].apply(lambda x : str(int(x)))

## Add State, County, and Census Block ID to Each Footprint

## Select Blocks within Bounding Box of Buildings

In [38]:
census_blocks_gdf['BLOCKID10'].describe()

count                9621
unique               9621
top       290970115006004
freq                    1
Name: BLOCKID10, dtype: object

In [39]:
# Find the bounds of the Builidngs to select Census Blocks
# Add Small Buffer for blocks on the edges
buffer = 0.001
minx = building_gdf.bounds.minx.min() + buffer
miny = building_gdf.bounds.miny.min() + buffer
maxx = building_gdf.bounds.maxx.max() + buffer
maxy = building_gdf.bounds.maxy.max() + buffer
building_gdf_bounds = [minx, miny, maxx, maxy]
building_gdf_bounds

[-94.58278849026384, 37.01648970170432, -94.40456812574384, 37.148296907577325]

In [40]:
# Select pumas within Bounds of Study Area
# build the r-tree index - for census blocks
sindex_census_blocks_gdf = census_blocks_gdf.sindex
possible_matches_index = list(sindex_census_blocks_gdf.intersection(building_gdf_bounds))
building_census_blocks_gdf = census_blocks_gdf.iloc[possible_matches_index]
building_census_blocks_gdf['BLOCKID10'].describe()

count                2958
unique               2958
top       291450205021050
freq                    1
Name: BLOCKID10, dtype: object

## Add Census Geogrpahy Details to Buildings

In [41]:
# Significant help from: https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
# Significant help from: https://github.com/gboeing/urban-data-science/blob/master/19-Spatial-Analysis-and-Cartography/rtree-spatial-indexing.ipynb
# build the r-tree index - Using Representative Point
building_gdf.loc[building_gdf.index,'geometry'] = building_gdf['rppnt4326']
sindex_building_gdf = building_gdf.sindex
sindex_building_gdf

In [42]:
# find the points that intersect with each subpolygon and add ID to Point
for index, block in building_census_blocks_gdf.iterrows():
    if index%100==0:
        print('.', end ="")

    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex_building_gdf.intersection(block['geometry'].bounds))
    possible_matches = building_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(block['geometry'])]
    building_gdf.loc[precise_matches.index,'BLOCKID10'] = block['BLOCKID10']
    building_gdf.loc[precise_matches.index,'STATEFP10'] = block['STATEFP10']
    building_gdf.loc[precise_matches.index,'COUNTYFP10'] = block['COUNTYFP10']
    building_gdf.loc[precise_matches.index,'TRACTCE10'] = block['TRACTCE10']
    building_gdf.loc[precise_matches.index,'PUMGEOID10'] = block['PUMGEOID10']
    building_gdf.loc[precise_matches.index,'PUMNAME10'] = block['PUMNAME10']
    building_gdf.loc[precise_matches.index,'PLCGEOID10'] = block['PLCGEOID10']
    building_gdf.loc[precise_matches.index,'PLCNAME10'] = block['PLCNAME10']
    building_gdf.loc[precise_matches.index,'HOUSING10'] = block['HOUSING10']
    building_gdf.loc[precise_matches.index,'apcount'] = block['apcount']
    building_gdf.loc[precise_matches.index,'gqpop10'] = block['gqpop10']
    building_gdf.loc[precise_matches.index,'POP10'] = block['POP10']

...................................

In [43]:
pd.crosstab(index=building_gdf['COUNTYFP10'], columns="count", margins=True, margins_name="Total")

col_0,count,Total
COUNTYFP10,,
97.0,24583,24583
145.0,3569,3569
Total,28152,28152


In [44]:
# Move Foriegn Key Columns Block ID State, County, Tract to first Columns
first_columns = ['bldg_id','BLOCKID10','STATEFP10','COUNTYFP10','TRACTCE10','PUMGEOID10','PUMNAME10','PLCGEOID10','PLCNAME10']
cols = first_columns + [col for col in building_gdf if col not in first_columns]
building_gdf = building_gdf[cols]
building_gdf.head()

,bldg_id,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10,ADDRESS1,...,x,y,geometry,rppnt4326,bply4326,gsq_meter,HOUSING10,apcount,gqpop10,POP10
0,B0000001,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,2519 N HIGHVIEW AVE,...,-94.477334,37.111732,POINT (-94.47732618279426 37.1117474687169),POINT (-94.47732618279426 37.1117474687169),POLYGON ((-94.47739972933748 37.11177310447127...,107.940427,103.0,103.0,0.0,226.0
1,B0000002,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,2515 N HIGHVIEW AVE,...,-94.477190,37.111399,POINT (-94.47719019368949 37.1114010706726),POINT (-94.47719019368949 37.1114010706726),"POLYGON ((-94.4771326960593 37.11131771353134,...",169.494274,103.0,103.0,0.0,226.0
2,B0000003,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,2511 N HIGHVIEW AVE,...,-94.476820,37.111433,POINT (-94.47682036688217 37.11143251198735),POINT (-94.47682036688217 37.11143251198735),POLYGON ((-94.47689239616878 37.11139956669546...,93.622453,103.0,103.0,0.0,226.0
3,B0000004,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,2511 N HIGHVIEW AVE,...,-94.477223,37.111174,POINT (-94.47722272093182 37.11117421422948),POINT (-94.47722272093182 37.11117421422948),POLYGON ((-94.47727628758035 37.11108068083621...,185.178358,103.0,103.0,0.0,226.0
4,B0000005,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,2501 N HIGHVIEW AVE,...,-94.477309,37.110922,POINT (-94.47730922150966 37.11092173624859),POINT (-94.47730922150966 37.11092173624859),"POLYGON ((-94.47736583467571 37.1108311087985,...",202.291430,103.0,103.0,0.0,226.0


In [45]:
building_gdf.crs

{'init': 'epsg:4326'}

In [46]:
# Switch Geometry Back to Footprint polygon
building_gdf.loc[building_gdf.index,'geometry'] = building_gdf['bply4326']

### How many buildings do not have block id information?

In [47]:
building_gdf.loc[(building_gdf['BLOCKID10'].isnull())]

,bldg_id,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10,ADDRESS1,...,x,y,geometry,rppnt4326,bply4326,gsq_meter,HOUSING10,apcount,gqpop10,POP10


## Add Parcel ID and parcel data to Each Footprint

## Read in Parcel Data
Parcel data polygons cleaned in an earlier program.

In [53]:
source_program = "IN-CORE_1bv2_Joplin_CleanParcelData_2019-07-11"
parcel_csv = source_program+"/"+source_program+"_EPSG4326.csv"
parcel_df = pd.read_csv(parcel_csv)
parcel_df.head()

C:\Users\nathanael99\AppData\Local\Continuum\anaconda3\envs\pyincore20190618\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,parid,geometry,PIN,Graphic_Ac,Legal_Ac,Address,Notes,Zoning,Own_Name,...,classv2,taxsheetv2,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10
0,0,P01401700000001000,POLYGON ((-94.13699017271023 37.35055489878356...,1.4017e+15,159.58900,160.00,0 THORN & COUNTY RD 50,NaN,NaN,"SEELA, RAYMOND E & FLORENCE A",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
1,1,P01401700000002000,POLYGON ((-94.14188721000529 37.33973595930397...,1.4017e+15,161.07700,158.22,5296 THORN RD,NaN,NaN,"NEIDIGH, JOSEPH A & LISA L",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
2,2,P01401700000003000,POLYGON ((-94.14613735909428 37.35070183976294...,1.4017e+15,323.00700,317.32,5699 THORN RD,NaN,NaN,"WILSON, SAM L & KRISTI L",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
3,3,P01401700000004000,POLYGON ((-94.14660779875197 37.33704609240082...,1.4017e+15,2.47968,2.48,5503 THORN RD,NaN,NaN,PLEASANT VIEW CEMETERY ASSN,...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
4,4,P01401800000001000,POLYGON ((-94.15538782377411 37.35084969271262...,1.4018e+15,313.29600,313.50,20912 COUNTY RD 70,NaN,NaN,"POTTS, LOREN A JR & GEORGIA H",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN


In [54]:
parcel_gdf = gpd.GeoDataFrame(parcel_df)
parcel_gdf.head()

,Unnamed: 0,parid,geometry,PIN,Graphic_Ac,Legal_Ac,Address,Notes,Zoning,Own_Name,...,classv2,taxsheetv2,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10
0,0,P01401700000001000,POLYGON ((-94.13699017271023 37.35055489878356...,1.4017e+15,159.58900,160.00,0 THORN & COUNTY RD 50,NaN,NaN,"SEELA, RAYMOND E & FLORENCE A",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
1,1,P01401700000002000,POLYGON ((-94.14188721000529 37.33973595930397...,1.4017e+15,161.07700,158.22,5296 THORN RD,NaN,NaN,"NEIDIGH, JOSEPH A & LISA L",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
2,2,P01401700000003000,POLYGON ((-94.14613735909428 37.35070183976294...,1.4017e+15,323.00700,317.32,5699 THORN RD,NaN,NaN,"WILSON, SAM L & KRISTI L",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
3,3,P01401700000004000,POLYGON ((-94.14660779875197 37.33704609240082...,1.4017e+15,2.47968,2.48,5503 THORN RD,NaN,NaN,PLEASANT VIEW CEMETERY ASSN,...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
4,4,P01401800000001000,POLYGON ((-94.15538782377411 37.35084969271262...,1.4018e+15,313.29600,313.50,20912 COUNTY RD 70,NaN,NaN,"POTTS, LOREN A JR & GEORGIA H",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN


In [55]:
parcel_gdf['geometry'].geom_type.describe()

count     0
unique    0
dtype: int64

In [56]:
# Use shapely.wkt loads to convert WKT to GeoSeries
from shapely.wkt import loads

parcel_gdf['geometry'] = parcel_gdf['geometry'].apply(lambda x: loads(x))
parcel_gdf['geometry'].geom_type.describe()

count       57235
unique          2
top       Polygon
freq        56786
dtype: object

In [57]:
parcel_gdf = parcel_gdf.set_geometry(parcel_gdf['geometry'])
parcel_gdf.crs = {'init':'epsg:4326'}
parcel_gdf.head()

,Unnamed: 0,parid,geometry,PIN,Graphic_Ac,Legal_Ac,Address,Notes,Zoning,Own_Name,...,classv2,taxsheetv2,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10
0,0,P01401700000001000,POLYGON ((-94.13699017271023 37.35055489878356...,1.4017e+15,159.58900,160.00,0 THORN & COUNTY RD 50,NaN,NaN,"SEELA, RAYMOND E & FLORENCE A",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
1,1,P01401700000002000,POLYGON ((-94.14188721000529 37.33973595930397...,1.4017e+15,161.07700,158.22,5296 THORN RD,NaN,NaN,"NEIDIGH, JOSEPH A & LISA L",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
2,2,P01401700000003000,POLYGON ((-94.14613735909428 37.35070183976294...,1.4017e+15,323.00700,317.32,5699 THORN RD,NaN,NaN,"WILSON, SAM L & KRISTI L",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
3,3,P01401700000004000,POLYGON ((-94.14660779875197 37.33704609240082...,1.4017e+15,2.47968,2.48,5503 THORN RD,NaN,NaN,PLEASANT VIEW CEMETERY ASSN,...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN
4,4,P01401800000001000,POLYGON ((-94.15538782377411 37.35084969271262...,1.4018e+15,313.29600,313.50,20912 COUNTY RD 70,NaN,NaN,"POTTS, LOREN A JR & GEORGIA H",...,NaN,NaN,2.909701e+14,29.0,97.0,12100.0,2902800.0,Jasper & Newton Counties PUMA,NaN,NaN


In [58]:
# Check CRS for Building Centroid and Block
parcel_gdf.crs

{'init': 'epsg:4326'}

In [59]:
building_gdf.crs

{'init': 'epsg:4326'}

In [60]:
census_blocks_gdf.geometry.name

'geometry'

## Select Parcels within Bounding Box of Buildings

In [61]:
parcel_gdf['parid'].describe()

count                  57235
unique                 57235
top       P19501630005009000
freq                       1
Name: parid, dtype: object

In [62]:
# Find the bounds of the Builidngs to select Census Blocks
# Add Small Buffer for blocks on the edges
buffer = 0.001
minx = building_gdf.bounds.minx.min() + buffer
miny = building_gdf.bounds.miny.min() + buffer
maxx = building_gdf.bounds.maxx.max() + buffer
maxy = building_gdf.bounds.maxy.max() + buffer
building_gdf_bounds = [minx, miny, maxx, maxy]
building_gdf_bounds

[-94.58278849026384, 37.01648970170432, -94.40456812574384, 37.148296907577325]

In [63]:
# Select parcels within Bounds of Study Area
# build the r-tree index - for census blocks
sindex_parcel_gdf  = parcel_gdf.sindex
possible_matches_index = list(sindex_parcel_gdf.intersection(building_gdf_bounds))
building_parcel_gdf = parcel_gdf.iloc[possible_matches_index]
building_parcel_gdf['parid'].describe()

count                  28503
unique                 28503
top       P19501630005009000
freq                       1
Name: parid, dtype: object

## Add Parcel Data to Buildings

In [64]:
# Significant help from: https://geoffboeing.com/2016/10/r-tree-spatial-index-python/
# Significant help from: https://github.com/gboeing/urban-data-science/blob/master/19-Spatial-Analysis-and-Cartography/rtree-spatial-indexing.ipynb
# build the r-tree index - Using Representative Point
building_gdf.loc[building_gdf.index,'geometry'] = building_gdf['rppnt4326']
sindex_building_gdf = building_gdf.sindex
sindex_building_gdf

In [65]:
# find the points that intersect with each subpolygon and add ID to Point
for index, parcel in building_parcel_gdf.iterrows():
    if index%100==0:
        print('.', end ="")

    # find approximate matches with r-tree, then precise matches from those approximate ones
    possible_matches_index = list(sindex_building_gdf.intersection(parcel['geometry'].bounds))
    possible_matches = building_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(parcel['geometry'])]
    building_gdf.loc[precise_matches.index,'parid'] = parcel['parid']
    building_gdf.loc[precise_matches.index,'Zoning'] = parcel['Zoning']
    building_gdf.loc[precise_matches.index,'APRBLDG'] = parcel['APRBLDG']
    building_gdf.loc[precise_matches.index,'APRLAND'] = parcel['APRLAND']
    building_gdf.loc[precise_matches.index,'APRTOT'] = parcel['APRTOT']
    building_gdf.loc[precise_matches.index,'CLASS'] = parcel['CLASS']
    building_gdf.loc[precise_matches.index,'STRUCTURE'] = parcel['STRUCTURE']
    building_gdf.loc[precise_matches.index,'YRBLT'] = parcel['YRBLT']
    building_gdf.loc[precise_matches.index,'classv2'] = parcel['classv2']
    building_gdf.loc[precise_matches.index,'taxsheetv2'] = parcel['taxsheetv2']

.............................................................................................................................................................................................................................................................................................

In [66]:
pd.crosstab(index=building_gdf['Zoning'], columns="count", margins=True, margins_name="Total")

col_0,count,Total
Zoning,,
C1,316,316
C1-PD,40,40
C2,184,184
C2-PD,25,25
C3,497,497
C3-PD,23,23
CO,146,146
CO-PD,32,32
M1,14,14


In [67]:
# Move Foriegn Key Columns Block ID State, County, Tract to first Columns
first_columns = ['bldg_id','parid']
cols = first_columns + [col for col in building_gdf if col not in first_columns]
building_gdf = building_gdf[cols]
building_gdf.head()

,bldg_id,parid,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10,...,POP10,Zoning,APRBLDG,APRLAND,APRTOT,CLASS,STRUCTURE,YRBLT,classv2,taxsheetv2
0,B0000001,P16703610001025000,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,226.0,R1,43070.0,11770.0,54840.0,NaN,NaN,1945.0,NaN,2010 Res
1,B0000002,P16703610001028000,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,226.0,R1,62810.0,9450.0,72260.0,NaN,NaN,1966.0,NaN,2010 Res
2,B0000003,P16703610001027000,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,226.0,CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B0000004,P16703610001029000,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,226.0,R1,77220.0,8890.0,86110.0,NaN,NaN,1965.0,NaN,2010 Res
4,B0000005,P16703610001059000,290970102003000,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,226.0,C1,48820.0,11470.0,60290.0,NaN,NaN,1955.0,NaN,2010 Res


In [68]:
building_gdf.crs

{'init': 'epsg:4326'}

In [69]:
# Switch Geometry Back to Footprint polygon
building_gdf.loc[building_gdf.index,'geometry'] = building_gdf['bply4326']

In [70]:
# Save Work at this point as CSV
savefile = sys.path[0]+"/"+programname+"/"+programname+".csv"
building_gdf.to_csv(savefile)

### How many buildings do not have Block ID

In [71]:
building_gdf.loc[building_gdf['BLOCKID10'].isnull()]

,bldg_id,parid,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10,...,POP10,Zoning,APRBLDG,APRLAND,APRTOT,CLASS,STRUCTURE,YRBLT,classv2,taxsheetv2


### How many buildings do not have Parcel ID - In Jasper County

In [72]:
building_gdf.loc[(building_gdf['parid'].isnull()) & (building_gdf['COUNTYFP10'] == 97)]

,bldg_id,parid,BLOCKID10,STATEFP10,COUNTYFP10,TRACTCE10,PUMGEOID10,PUMNAME10,PLCGEOID10,PLCNAME10,...,POP10,Zoning,APRBLDG,APRLAND,APRTOT,CLASS,STRUCTURE,YRBLT,classv2,taxsheetv2
654,B0000668,None,290970102002002,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
851,B0000871,None,290970102004018,29.0,97.0,10200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1586,B0016973,None,290970112004047,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,B0017094,None,290970112004043,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1702,B0017097,None,290970112004037,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1706,B0017101,None,290970112004037,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1721,B0017116,None,290970112004036,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1722,B0017117,None,290970112004036,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,B0017195,None,290970112004005,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1803,B0017199,None,290970112004005,29.0,97.0,11200.0,2902800.0,Jasper & Newton Counties PUMA,2937592.0,Joplin,...,112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Building B0000668
Located between parcels in a wooded area - probably not a building.

#### Building B0000871
Building center falls in between parcels where there is a gap.

#### Building B0017117
Building center falls just outside of parcel - main building is in the parcel.


### How Many Buildings do not have Appraised Values in Joplin?

In [73]:
pd.crosstab(building_gdf['COUNTYFP10'].loc[(building_gdf['PLCNAME10'] == 'Joplin')], columns="count", margins=True, margins_name="Total")

col_0,count,Total
COUNTYFP10,,
97.0,22339,22339
145.0,2664,2664
Total,25003,25003


In [74]:
pd.crosstab(building_gdf['COUNTYFP10'].loc[building_gdf['APRTOT'].isnull()], columns="count", margins=True, margins_name="Total")

col_0,count,Total
COUNTYFP10,,
97.0,1398,1398
145.0,3569,3569
Total,4967,4967


In [75]:
pd.crosstab(building_gdf['COUNTYFP10'].loc[(building_gdf['APRTOT'].isnull()) & 
                                           (building_gdf['PLCNAME10'] == 'Joplin')], 
                                           columns= "count", margins=True, margins_name="Total")

col_0,count,Total
COUNTYFP10,,
97.0,1053,1053
145.0,2664,2664
Total,3717,3717


#### Conclusion - 1,053 buildings in Joplin (out of 22,339 - 4.7%) do not have appraised building values. 

### How many parcels do not have buildings - in joplin - with Bldg Value

In [76]:
# Collapse Blocks By Place Name and Count Blocks 
building_gdf_bldgcount = building_gdf[['parid']]
building_gdf_bldgcount['prclbldsum'] = 1
building_gdf_bldgcount_sum = building_gdf_bldgcount.groupby(['parid']).sum()
building_gdf_bldgcount_sum['prclbldsum'].describe()

C:\Users\nathanael99\AppData\Local\Continuum\anaconda3\envs\pyincore20190618\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count    18409.000000
mean         1.330653
std          1.034296
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         61.000000
Name: prclbldsum, dtype: float64

In [77]:
# Add Building Count to Parcel Data
parcel_gdf_checkbuilding_count = pd.merge(parcel_gdf, building_gdf_bldgcount_sum,
                                  left_on='parid', right_on='parid', how='left')
displaycols = ['parid','APRBLDG']
parcel_gdf_checkbuilding_count[displaycols].loc[(parcel_gdf_checkbuilding_count['prclbldsum'].isnull()) & 
                                      (parcel_gdf_checkbuilding_count['COUNTYFP10'] == 97) &
                                      (parcel_gdf_checkbuilding_count['APRBLDG'] > 500000) & 
                                      (parcel_gdf_checkbuilding_count['PLCNAME10'] == 'Joplin')].sort_values(by=['APRBLDG'])

,parid,APRBLDG
49838,P19601310013009000,540130.0
53905,P20300720012001000,583360.0
39154,P19101140031004000,595230.0
46150,P19401700000005020,597970.0
43024,P19200910014002001,607560.0
53904,P20300720011002000,644690.0
46153,P19401700000005023,681330.0
55056,P20501500000018009,699980.0
53474,P20300630001032011,736610.0
50740,P19601410001007000,759750.0


In [78]:
# Total tax values not included
total_tax_value = parcel_gdf_checkbuilding_count['APRTOT'].loc[(parcel_gdf_checkbuilding_count['COUNTYFP10'] == 97) &
                                      (parcel_gdf_checkbuilding_count['PLCNAME10'] == 'Joplin')].sum()
total_tax_value_missing = parcel_gdf_checkbuilding_count['APRTOT'].loc[(parcel_gdf_checkbuilding_count['prclbldsum'].isnull()) & 
                                      (parcel_gdf_checkbuilding_count['COUNTYFP10'] == 97) &
                                      (parcel_gdf_checkbuilding_count['PLCNAME10'] == 'Joplin')].sum()
percent_difference = total_tax_value_missing / total_tax_value
print("The total appraised value of parcels in Joplin was $", format(total_tax_value, ',.2f'))
print("The total appraised value of parcels missing buildings in Joplin was $", format(total_tax_value_missing, ',.2f'))
print("The percent difference was", format(percent_difference*100, ',.2f'))

The total appraised value of parcels in Joplin was $ 1,731,289,490.00
The total appraised value of parcels missing buildings in Joplin was $ 96,669,130.00
The percent difference was 5.58


### The missmatch between buildings and parcels adds up to 96.7 Million Dollars

### Parcel P19200340019001000 - 3.2 Million
The building covers 2 parcels and the represetnative point falls into another parcel (P19100230016001000). Parcel P19100230016001000 does not have any tax information.
I think this is a good example of needing to split buildings by parcels. However, this is also the case where the building is drawn in such a way that parts of the building would fall into 4 parcels - even though it only falls into 1 parcel.

### Parcel P20300600000001001 - 2.9 Million
This is the mall which is one large building but covers multiple parcels.

### Parcel P15903000000036000 - 2.2 Million
Large industrial building that covers multiple parcels.

### Parcel P19601310013009000 - 540K
Building Footprint appears to be missing in this case.

## Conlcusion - Need to address issue of mismatch between large buildings and parcel boundaries. 
The .union option should address this issue. The overlay union command should split buildings by parcel boundaries. This will allow for large buildings to be represented by multiple polygons that line up with the parcel boundaries.

### Create a list of builidngs that are missing based on parcel data


In [80]:
parcels_withoutbuildings = parcel_gdf_checkbuilding_count.loc[(parcel_gdf_checkbuilding_count['prclbldsum'].isnull()) & 
                                                              (parcel_gdf_checkbuilding_count['APRBLDG']>0)]
pd.crosstab(parcels_withoutbuildings['PLCNAME10'], parcels_withoutbuildings['COUNTYFP10'], margins=True, margins_name="Total")

COUNTYFP10,97.0,Total
PLCNAME10,,
Airport Drive,242,242
Avilla,54,54
Brooklyn Heights,42,42
Carl Junction,2595,2595
Carterville,749,749
Carthage,5074,5074
Duenweg,430,430
Duquesne,112,112
Fidelity,60,60


In [81]:
parcels_withoutbuildings.columns

Index(['Unnamed: 0', 'parid', 'geometry', 'PIN', 'Graphic_Ac', 'Legal_Ac',
       'Address', 'Notes', 'Zoning', 'Own_Name', 'Own_Addres', 'SHAPE_Leng',
       'SHAPE_Area', 'rppnt4326', 'prcl4326', 'ACRES', 'ADRDIR', 'ADRNO',
       'ADRSTR', 'APRBLDG', 'APRLAND', 'APRTOT', 'CARD', 'CITYNAME', 'CLASS',
       'OWN1', 'PARID', 'STATECODE', 'STRUCTURE', 'TAXYR', 'YRBLT', 'ZIP1',
       'taxsheet', 'parcel_count', 'classv2', 'taxsheetv2', 'BLOCKID10',
       'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'PUMGEOID10', 'PUMNAME10',
       'PLCGEOID10', 'PLCNAME10', 'prclbldsum'],
      dtype='object')

### For parcels in Joplin it may be possible to add building ID to parcel

In [114]:
sindex_building_gdf = building_gdf.sindex

# find the points that intersect with each subpolygon and add ID to Point
for index, parcel in parcels_withoutbuildings_joplin.iterrows():
    if index%100==0:
        print('.', end ="")
    
    # find approximate matches with r-, then precise matches from those approximate ones
    possible_matches_index = list(sindex_building_gdf.intersection(parcel['geometry'].bounds))
    possible_matches = building_gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.intersects(parcel['geometry'])]
    # print("ParcelID = ",parcel['parid']," Builidng ID" ,precise_matches['bldg_id'])
    parcels_withoutbuildings_joplin.loc[parcels_withoutbuildings_joplin['parid']==parcel['parid']
                                       ,'bldg_id'] = str(precise_matches['bldg_id'].values)

.....

In [124]:
parcels_withoutbuildings_joplin[['parid','bldg_id']].loc[parcels_withoutbuildings_joplin['bldg_id']!="[]"].describe()

,parid,bldg_id
count,219,219
unique,219,158
top,P19100220011009000,['B0020351']
freq,1,9


In [140]:
parcels_withoutbuildings_joplinv2 = parcels_withoutbuildings_joplin.loc[parcels_withoutbuildings_joplin['bldg_id']=="[]"]
keepcols = ['bldg_id','parid','Address','COUNTYFP10','BLOCKID10','PLCNAME10','rppnt4326','Zoning', 'APRBLDG', 'APRLAND', 'APRTOT', 'CLASS', 'STRUCTURE', 'YRBLT', 'classv2', 'taxsheetv2']
parcels_withoutbuildings_joplinv2 = parcels_withoutbuildings_joplinv2[keepcols]
parcels_withoutbuildings_joplinv2['source'] = "Jasper County Parcel Shapefile"
parcels_withoutbuildings_joplinv2[['bldg_id','parid','PLCNAME10']].describe()

,bldg_id,parid,PLCNAME10
count,299,299,299
unique,1,299,1
top,[],P19401700000005020,Joplin
freq,299,1,299


In [145]:
parcels_withoutbuildings_joplinv2.columns

Index(['bldg_id', 'parid', 'Address', 'COUNTYFP10', 'BLOCKID10', 'PLCNAME10',
       'rppnt4326', 'Zoning', 'APRBLDG', 'APRLAND', 'APRTOT', 'CLASS',
       'STRUCTURE', 'YRBLT', 'classv2', 'taxsheetv2', 'source'],
      dtype='object')

### Append Parcels with Missing Building Data with Building Inventory

In [141]:
building_count = building_gdf['bldg_id'].count()
parcel_count   = parcels_withoutbuildings_joplinv2['bldg_id'].count()
expected_append_count = building_count + parcel_count
print("The combined data will have ", building_count, "+",parcel_count," = ", expected_append_count)

The combined data will have  28152 + 299  =  28451


In [142]:
building_gdf_parceladd = building_gdf.append(parcels_withoutbuildings_joplinv2, sort=True)
building_gdf_parceladd['bldg_id'].count()

28451

In [143]:
building_gdf_parceladd.columns

Index(['ADDRESS1', 'APRBLDG', 'APRLAND', 'APRTOT', 'Address', 'BLOCKID10',
       'CLASS', 'COUNTYFP10', 'HOUSE_NO', 'HOUSING10', 'PLCGEOID10',
       'PLCNAME10', 'POP10', 'PUMGEOID10', 'PUMNAME10', 'STATEFP10',
       'STRUCTURE', 'TRACTCE10', 'YRBLT', 'Zoning', 'a_stories', 'apcount',
       'appr_bldg', 'archtype', 'b_stories', 'bldg_id', 'bply4326',
       'broad_occ', 'bsmt_type', 'classv2', 'cont_val', 'dgn_lvl',
       'dwell_unit', 'efacility', 'geometry', 'gqpop10', 'gsq_foot',
       'gsq_meter', 'guid', 'major_occ', 'no_stories', 'nstra_cst',
       'nstrd_cst', 'objectid', 'occ_detail', 'occ_typ2', 'occ_type', 'parid',
       'parid_card', 'repl_cst', 'rppnt4326', 'source', 'sq_foot', 'str_cst',
       'str_prob', 'str_typ2', 'struct_typ', 'taxsheetv2', 'tract_id', 'x',
       'y', 'year_built'],
      dtype='object')

In [161]:
building_gdf_parceladd[['BLOCKID10','parid','source']].loc[building_gdf_parceladd['source']=='Jasper County Parcel Shapefile'].head()

,BLOCKID10,parid,source
28152,2.9097e+14,P15903110001024015,Jasper County Parcel Shapefile
28153,2.9097e+14,P15903120006002002,Jasper County Parcel Shapefile
28154,2.9097e+14,P15903130006024010,Jasper County Parcel Shapefile
28155,2.9097e+14,P15903140002002000,Jasper County Parcel Shapefile
28156,2.9097e+14,P15903140002012000,Jasper County Parcel Shapefile


### How many blocks that have housing units do not have buildings?

In [194]:
# Collapse Blocks By Place Name and Count Blocks 
building_gdf_blockbldgcount = building_gdf_parceladd[['BLOCKID10']]
building_gdf_blockbldgcount['blockbldsum'] = 1
building_gdf_blockbldsum = building_gdf_blockbldgcount.groupby(['BLOCKID10']).sum()
building_gdf_blockbldsum['blockbldsum'].describe()

C:\Users\nathanael99\AppData\Local\Continuum\anaconda3\envs\pyincore20190618\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count    1987.000000
mean       14.318571
std        16.297451
min         1.000000
25%         5.000000
50%        12.000000
75%        18.000000
max       281.000000
Name: blockbldsum, dtype: float64

In [195]:
building_gdf_blockbldsum['blockid10'] = building_gdf_blockbldsum.index
building_gdf_blockbldsum['blockid10'] = building_gdf_blockbldsum['blockid10'].apply(lambda x : str((int(x))))
building_gdf_blockbldsum.head()

,blockbldsum,blockid10
BLOCKID10,,
2.909701e+14,1,290970101001008
2.909701e+14,2,290970101001039
2.909701e+14,2,290970101001041
2.909701e+14,1,290970101001048
2.909701e+14,1,290970101001064


In [196]:
building_gdf_blockbldsum.loc[building_gdf_blockbldsum['blockid10']=='290970109001063']

,blockbldsum,blockid10
BLOCKID10,,
2.909701e+14,11,290970109001063


In [197]:
census_blocks_gdf.loc[census_blocks_gdf['BLOCKID10']=='290970109001063']

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,...,blk104269,blockid,apcount,pop10,gqpop10,popdiff,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10
5359,5359,29,97,10900,1063,290970109001063,N,10,20,"POLYGON ((-94.57345099999999 37.059648, -94.57...",...,"POLYGON ((-94.57345099999999 37.059648, -94.57...",2.909701e+14,10.0,20.0,0.0,0.0,2937592.0,Joplin,2902800,Jasper & Newton Counties PUMA


In [198]:
# Add Building Count to Block Data
census_blocks_gdf_checkbuilding_count = pd.merge(census_blocks_gdf, building_gdf_blockbldsum, 
                                                 left_on='BLOCKID10', right_on='blockid10', how='left')
census_blocks_gdf_checkbuilding_count.loc[census_blocks_gdf_checkbuilding_count['BLOCKID10']=='290970109001063']

,Unnamed: 0,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE,BLOCKID10,PARTFLG,HOUSING10,POP10,geometry,...,apcount,pop10,gqpop10,popdiff,PLCGEOID10,PLCNAME10,PUMGEOID10,PUMNAME10,blockbldsum,blockid10
5506,5359,29,97,10900,1063,290970109001063,N,10,20,"POLYGON ((-94.57345099999999 37.059648, -94.57...",...,10.0,20.0,0.0,0.0,2937592.0,Joplin,2902800,Jasper & Newton Counties PUMA,11.0,290970109001063


In [199]:
displaycols = ['BLOCKID10','apcount']
blocks_withoutbuildings = census_blocks_gdf_checkbuilding_count.loc[(census_blocks_gdf_checkbuilding_count['blockbldsum'].isnull()) & 
                                                       (census_blocks_gdf_checkbuilding_count['apcount']>0)]
pd.crosstab(blocks_withoutbuildings['PLCNAME10'], blocks_withoutbuildings['COUNTYFP10'], margins=True, margins_name="Total")


COUNTYFP10,97,145,Total
PLCNAME10,,,
Airport Drive,19,0,19
Alba,30,0,30
Asbury,31,0,31
Avilla,10,0,10
Brooklyn Heights,9,0,9
Carl Junction,150,0,150
Carterville,104,0,104
Carthage,439,0,439
Carytown,24,0,24


### Create list of buildings to add to building inventory based on Census Block Data
The building inventory could be improved by adding building observations based on the Census Block Data. If the Census Block indicates that there should be housing units in the block then this information could be used to identify missing buildings in the existing inventory. 

In [200]:
missing_buildings_blocks_pd = pd.DataFrame(np.repeat(blocks_withoutbuildings.values,blocks_withoutbuildings['apcount'],axis=0))
missing_buildings_blocks_pd.columns = blocks_withoutbuildings.columns
missing_buildings_blocks_pd[['BLOCKID10','apcount','rppnt4269']].head(10)

,BLOCKID10,apcount,rppnt4269
0,290970121001047,2,POINT (-94.14712692867573 37.320531)
1,290970121001047,2,POINT (-94.14712692867573 37.320531)
2,290970121001050,3,POINT (-94.12889453205761 37.316266)
3,290970121001050,3,POINT (-94.12889453205761 37.316266)
4,290970121001050,3,POINT (-94.12889453205761 37.316266)
5,290970121001094,4,POINT (-94.22108805775474 37.302019)
6,290970121001094,4,POINT (-94.22108805775474 37.302019)
7,290970121001094,4,POINT (-94.22108805775474 37.302019)
8,290970121001094,4,POINT (-94.22108805775474 37.302019)
9,290970121001093,5,POINT (-94.20604827951779 37.2853095)


In [201]:
pd.crosstab(missing_buildings_blocks_pd['PLCNAME10'], missing_buildings_blocks_pd['COUNTYFP10'], margins=True, margins_name="Total")

COUNTYFP10,97,145,Total
PLCNAME10,,,
Airport Drive,243,0,243
Alba,251,0,251
Asbury,103,0,103
Avilla,54,0,54
Brooklyn Heights,52,0,52
Carl Junction,2770,0,2770
Carterville,800,0,800
Carthage,5760,0,5760
Carytown,109,0,109


In [202]:
missing_buildings_blocks_pd['source'] = "2010 Census Block Data"

In [203]:
# Save Work at this point as CSV
savefile = sys.path[0]+"/"+programname+"/"+programname+"missing_buildings_blocks_pd.csv"
missing_buildings_blocks_pd.to_csv(savefile)

### The Missing Buildings File Should be added to the address point inventory
In exploring the blocks with missing parcel data and missing building data in Jasper County - the blocks look like they have housing units in error - the population should probably be assigned to another block nearby or within the Census Tract.

### Add Builiding Count By Parcel
The number of buildings on a parcel can be used to identify buildings that are more likely to be the primary address point. For example, if a parcel has 2 buildings and 1 building is a garage and the 1 building is a house the garage will not have an address point.

In [169]:
# Add Building Count Parcel to buildingdata
building_gdf_parceladd = pd.merge(building_gdf_parceladd, building_gdf_bldgcount_sum,
                                  left_on='parid', right_on='parid', how='left')
building_gdf_parceladd[['bldg_id','parid']].describe()

,bldg_id,parid
count,28451,24795
unique,28153,18708
top,[],P16702620002006000
freq,299,61


### Save Work

In [170]:
building_gdf_parceladd.head()

,ADDRESS1,APRBLDG,APRLAND,APRTOT,Address,BLOCKID10,CLASS,COUNTYFP10,HOUSE_NO,HOUSING10,...,str_prob,str_typ2,struct_typ,taxsheetv2,tract_id,x,y,year_built,prclbldsum_x,prclbldsum_y
0,2519 N HIGHVIEW AVE,43070.0,11770.0,54840.0,NaN,290970102003000,NaN,97.0,2519,103.0,...,None,None,None,2010 Res,None,-94.477334,37.111732,None,1.0,1.0
1,2515 N HIGHVIEW AVE,62810.0,9450.0,72260.0,NaN,290970102003000,NaN,97.0,2515,103.0,...,None,None,None,2010 Res,None,-94.477190,37.111399,None,1.0,1.0
2,2511 N HIGHVIEW AVE,NaN,NaN,NaN,NaN,290970102003000,NaN,97.0,2511,103.0,...,None,None,None,NaN,None,-94.476820,37.111433,None,1.0,1.0
3,2511 N HIGHVIEW AVE,77220.0,8890.0,86110.0,NaN,290970102003000,NaN,97.0,2511,103.0,...,None,None,None,2010 Res,None,-94.477223,37.111174,None,1.0,1.0
4,2501 N HIGHVIEW AVE,48820.0,11470.0,60290.0,NaN,290970102003000,NaN,97.0,2501,103.0,...,None,None,None,2010 Res,None,-94.477309,37.110922,None,1.0,1.0


In [171]:
# Check Columns
cols = [col for col in building_gdf_parceladd]
cols

['ADDRESS1',
 'APRBLDG',
 'APRLAND',
 'APRTOT',
 'Address',
 'BLOCKID10',
 'CLASS',
 'COUNTYFP10',
 'HOUSE_NO',
 'HOUSING10',
 'PLCGEOID10',
 'PLCNAME10',
 'POP10',
 'PUMGEOID10',
 'PUMNAME10',
 'STATEFP10',
 'STRUCTURE',
 'TRACTCE10',
 'YRBLT',
 'Zoning',
 'a_stories',
 'apcount',
 'appr_bldg',
 'archtype',
 'b_stories',
 'bldg_id',
 'bply4326',
 'broad_occ',
 'bsmt_type',
 'classv2',
 'cont_val',
 'dgn_lvl',
 'dwell_unit',
 'efacility',
 'geometry',
 'gqpop10',
 'gsq_foot',
 'gsq_meter',
 'guid',
 'major_occ',
 'no_stories',
 'nstra_cst',
 'nstrd_cst',
 'objectid',
 'occ_detail',
 'occ_typ2',
 'occ_type',
 'parid',
 'parid_card',
 'repl_cst',
 'rppnt4326',
 'source',
 'sq_foot',
 'str_cst',
 'str_prob',
 'str_typ2',
 'struct_typ',
 'taxsheetv2',
 'tract_id',
 'x',
 'y',
 'year_built',
 'prclbldsum_x',
 'prclbldsum_y']

In [172]:
# Save Work at this point as CSV
savefile = sys.path[0]+"/"+programname+"/"+programname+"_EPSG4326.csv"
building_gdf_parceladd.to_csv(savefile)